# Agentic RAG for Game Recommendation - Complete Tutorial

Welcome! This notebook will guide you through building an intelligent agent that can understand complex queries and provide smart game recommendations.

## What You'll Learn:
1. **Agentic RAG Concepts**: How agents think and reason
2. **Tool Integration**: Connecting your existing retrieval systems
3. **Local Model Setup**: Running everything offline
4. **Advanced Query Handling**: Multi-step reasoning and context awareness

## Prerequisites:
- Your existing RAG system (from reranking.ipynb)
- Basic Python knowledge
- Curiosity about AI agents! 🤖

## Step 1: Understanding Agentic RAG

### What's the difference between regular RAG and Agentic RAG?

**Regular RAG**: Query → Retrieve → Generate
- Simple, linear process
- Limited reasoning

**Agentic RAG**: Query → Think → Plan → Retrieve → Reason → Generate
- Multi-step reasoning
- Can use multiple tools
- Self-correcting and adaptive

### Example:
- **Regular RAG**: "Find games like Don't Starve" → Direct similarity search
- **Agentic RAG**: "I want a survival game that's challenging but not too dark" → 
  1. Understands "survival" genre
  2. Searches for survival games
  3. Filters by difficulty and tone
  4. Explains why each recommendation fits

In [18]:
# First, let's install the required packages
# Run this cell to install LlamaIndex and other dependencies

# %pip install llama-index llama-index-llms-ollama llama-index-embeddings-huggingface
# %pip install ollama

print("✅ Packages installed successfully!")

✅ Packages installed successfully!


In [1]:
import chromadb
import polars as pl
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import re
import json
from typing import List, Dict, Any

from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core import ServiceContext

print("✅ All imports successful!")

✅ All imports successful!


## Step 2: Setting Up Your Local LLM

### Why Local Models?
- **Privacy**: Your data stays on your machine
- **Cost**: No API fees
- **Control**: Customize for your specific needs
- **Offline**: Works without internet

### Let's set up Ollama (easiest way to run local LLMs)


In [2]:
# First, you need to install Ollama on your system
# Go to: https://ollama.ai/download
# Then run these commands in your terminal:

llm = Ollama(
    model="qwen3:0.6b", 
    request_timeout=60.0,
    temperature=0.1,
    top_p=0.9,
    additional_kwargs={"tool_choice": "auto"}
)
print(f"✅ LLM loaded: {llm.model}")

✅ LLM loaded: qwen3:0.6b


## Step 3: Loading Your Existing RAG System

We'll reuse your existing retrieval and reranking components. This is the beauty of Agentic RAG - it builds on what you already have!


In [3]:
# Load your existing models and data
print("🔄 Loading your existing RAG system...")

# ChromaDB client
client = chromadb.PersistentClient(
    path="../chroma_db",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# Load models
desc_encoder = SentenceTransformer("../models/all-MiniLM-L6-v2")
reranker = CrossEncoder("../models/bge-reranker-base-crossencoder")
processor = AutoImageProcessor.from_pretrained("../models/dinov2-base")
viz_encoder = AutoModel.from_pretrained("../models/dinov2-base")

# Load metadata
metadata = pl.read_ndjson('../data/mobygames_index.jsonl')
critics_emb = pl.read_ndjson('../embeddings/critics_embeddings.jsonl')
critics_reviews = pl.read_ndjson('../data/mobygames_critic_reviews.jsonl')

# Get collections
desc_collection = client.get_or_create_collection("desc_embeddings")
critics_collection = client.get_or_create_collection("critics_embeddings")
cover_collection = client.get_or_create_collection("cover_embeddings")
screenshot_collection = client.get_or_create_collection("screenshot_embeddings")

print("✅ All components loaded successfully!")
print(f"📊 Metadata: {metadata.height} games")
print(f"📊 Critics: {critics_reviews.height} reviews")

🔄 Loading your existing RAG system...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ All components loaded successfully!
📊 Metadata: 188267 games
📊 Critics: 609000 reviews


In [4]:
# Utility functions
def clean_html(text):
    if hasattr(text, 'item'):
        text = text.item()
    return re.sub('<[^<]+?>', '', str(text))

def get_all_critic_quotes(game_id, critics_collection):
    # Get review IDs from ChromaDB for this game
    results = critics_collection.get(
        where={"game_id": str(game_id)},
        include=[]
    )
    quotes = []
    for review_id in results["ids"]:
        # Find the actual review text in the critics_reviews DataFrame
        critic_row = critics_reviews.filter(pl.col("review_id") == int(review_id))
        if critic_row.height > 0:
            quotes.append(clean_html(critic_row[0]["citation"]) if "citation" in critic_row.columns else "")
    return quotes

def format_game_result(game_data, score=None):
    """Format a game result for display"""
    result = f"🎮 {game_data['title']}"
    if score:
        result += f" (Score: {score:.2f})"
    result += f"\n📝 {game_data['description'][:200]}..."
    if game_data.get('critic'):
        result += f"\n⭐ Critics: {game_data['critic'][:100]}..."
    return result

print("✅ Utility functions ready!")


✅ Utility functions ready!


## Step 4: Creating Agent Tools

### What are Tools?
Tools are functions that your agent can call to perform specific tasks. Think of them as the agent's "hands" - it can use them to interact with your data.

### We'll create 3 main tools:
1. **Text Search Tool**: Find games by description
2. **Image Search Tool**: Find games by visual similarity  
3. **Filter Tool**: Apply additional filters (genre, platform, etc.)


In [5]:
# Tool 1: Text Search
def text_search_tool(query: str, num_results: int = 10) -> str:
    """
    Search for games using text descriptions and reviews.
    
    Args:
        query: Text description of what you're looking for
        num_results: Number of results to return (default: 10)
    
    Returns:
        Formatted string with game recommendations
    """
    print(f"🔍 Searching for: '{query}'")
    
    # Get embeddings for both descriptions and critics
    query_embedding = desc_encoder.encode(query)
    
    # Search descriptions
    desc_results = desc_collection.query(
        query_embeddings=[query_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Search critics
    critics_results = critics_collection.query(
        query_embeddings=[query_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Combine and deduplicate
    candidates = []
    game_ids = set()
    
    for meta_list in [desc_results["metadatas"][0], critics_results["metadatas"][0]]:
        for meta in meta_list:
            game_id = int(meta["game_id"])
            if game_id not in game_ids:
                row = metadata.filter(pl.col("id") == game_id)
                if row.height > 0:
                    row = row[0]
                    critic_quotes = get_all_critic_quotes(game_id, critics_collection)
                    critic_text = " | ".join(critic_quotes) if critic_quotes else ""
                    candidates.append({
                        "game_id": game_id,
                        "title": clean_html(row["title"]),
                        "description": clean_html(row["description"]),
                        "critic": critic_text,
                    })
                    game_ids.add(game_id)
    
    # Rerank using cross-encoder
    pairs = []
    for c in candidates:
        desc = clean_html(c['description'])
        critic = clean_html(c['critic'])
        candidate_text = f"{c['title']}. {desc} Critics: {critic}"
        pairs.append([query, candidate_text])
    
    scores = reranker.predict(pairs, batch_size=16)
    reranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    
    # Format results
    result = f"Found {len(reranked)} games matching '{query}':\n\n"
    for i, (c, score) in enumerate(reranked[:num_results], 1):
        result += f"{i}. {format_game_result(c, score)}\n\n"
    
    return result

# Test the tool
test_result = text_search_tool("survival games with crafting", 3)
print("🧪 Testing text search tool:")
print(test_result)


🔍 Searching for: 'survival games with crafting'
🧪 Testing text search tool:
Found 2 games matching 'survival games with crafting':

1. 🎮 Craft the World (Score: 0.79)
📝 Craft The World is a 2D sandbox strategy game, combining the gameplay elements of Terraria and Dwarf Fortress into its own fantasy world, where a group of dwarves attempts to survive against onslaught...
⭐ Critics: Craft the World is a solid game.  Resources are plentiful and creatures are sporadic enough to reall...

2. 🎮 Craft (Score: 0.57)
📝 Craft is a puzzle game where the player is presented with a number of tiles where each piece has a part of a circuit. The objective is to create a complete circuit from an entry point to an exit so th...




In [6]:
# Tool 2: Image Search
def image_search_tool(image_path: str, num_results: int = 10) -> str:
    """
    Search for games using image similarity.
    
    Args:
        image_path: Path to the image file
        num_results: Number of results to return (default: 10)
    
    Returns:
        Formatted string with visually similar games
    """
    print(f"🖼️ Searching for games similar to: {image_path}")
    
    # Load and process image
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(viz_encoder.device) for k, v in inputs.items()}
    
    # Get image embedding
    with torch.no_grad():
        outputs = viz_encoder(**inputs)
        image_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    # Search both cover and screenshot collections
    cover_results = cover_collection.query(
        query_embeddings=[image_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    screenshot_results = screenshot_collection.query(
        query_embeddings=[image_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Combine results
    candidates = []
    game_ids = set()
    
    for meta_list in [cover_results["metadatas"][0], screenshot_results["metadatas"][0]]:
        for meta in meta_list:
            game_id = int(meta["game_id"])
            if game_id not in game_ids:
                row = metadata.filter(pl.col("id") == game_id)
                if row.height > 0:
                    row = row[0]
                    critic_quotes = get_all_critic_quotes(game_id, critics_collection)
                    critic_text = " | ".join(critic_quotes) if critic_quotes else ""
                    candidates.append({
                        "game_id": game_id,
                        "title": clean_html(row["title"]),
                        "description": clean_html(row["description"]),
                        "critic": critic_text,
                    })
                    game_ids.add(game_id)
    
    # Calculate similarity scores (normalize distances)
    cover_scores = {int(meta["game_id"]): 1 - dist for meta, dist in zip(cover_results["metadatas"][0], cover_results["distances"][0])}
    screenshot_scores = {int(meta["game_id"]): 1 - dist for meta, dist in zip(screenshot_results["metadatas"][0], screenshot_results["distances"][0])}
    
    # Score and rank
    for c in candidates:
        gid = c["game_id"]
        c["score"] = cover_scores.get(gid, 0) + screenshot_scores.get(gid, 0)
    
    reranked = sorted(candidates, key=lambda x: x["score"], reverse=True)
    
    # Format results
    result = f"Found {len(reranked)} visually similar games:\n\n"
    for i, c in enumerate(reranked[:num_results], 1):
        result += f"{i}. {format_game_result(c, c.get('score'))}\n\n"
    
    return result

# Test the tool (if you have a sample image)
try:
    test_result = image_search_tool("../data/sample.jpg", 3)
    print("🧪 Testing image search tool:")
    print(test_result)
except FileNotFoundError:
    print("⚠️ Sample image not found. Tool ready for use!")


🖼️ Searching for games similar to: ../data/sample.jpg
🧪 Testing image search tool:
Found 2 visually similar games:

1. 🎮 Don't Starve + Reign of Giants (Score: -1324.51)
📝 This edition includes:* Don't Starve (base game)

Don't Starve: Reign of Giants (DLC)

For the PlayStation Vita, PS3, Wii U, Xbox One and iOS this is the first and only release of the game. For PC and...
⭐ Critics: Also, the iPad version comes with the Rise of Giants DLC already included, which adds the Wetness ga...

2. 🎮 Spirits & Spells (Score: -1446.95)
📝 In Spirits and Spells the player takes the role of the two kids Alicia and Greg which visit a spooky old house with their friends. Unfortunately the Bogeyman steals the soles of their friends and turn...
⭐ Critics: Indeed I was starting to question if it had indeed been released at all. As a result it probably did...




## Step 5: Creating the Agent

### What is an Agent?
An agent is like a smart assistant that can:
- **Think** about what you're asking
- **Plan** how to find the best answer
- **Use tools** to search and filter
- **Reason** about the results
- **Explain** its recommendations

### The ReAct Pattern:
- **Reasoning**: Why am I doing this?
- **Acting**: What tool should I use?
- **Observing**: What did I learn?
- **Repeat** until satisfied


In [7]:
# Create the tools for our agent
tools = [
    FunctionTool.from_defaults(
        fn=text_search_tool,
        name="text_search",
        description="Search for games using text descriptions, genres, or themes"
    ),
    FunctionTool.from_defaults(
        fn=image_search_tool,
        name="image_search",
        description="Search for games using visual similarity to an uploaded image"
    )
]

print(f"✅ Created {len(tools)} tools for the agent:")
for tool in tools:
    print(f"  - {tool.metadata.name}: {tool.metadata.description}")


✅ Created 2 tools for the agent:
  - text_search: Search for games using text descriptions, genres, or themes
  - image_search: Search for games using visual similarity to an uploaded image


In [8]:
# Create the agent
print("🤖 Creating the game recommendation agent...")

# Set up the service context
embed_model = HuggingFaceEmbedding(
    model_name="../models/all-MiniLM-L6-v2"
)

from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

agent = ReActAgent(
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    streaming=False
)

print("✅ Agent created successfully!")

🤖 Creating the game recommendation agent...
✅ Agent created successfully!


## Step 6: Testing Your Agent

### Let's test different types of queries to see how your agent thinks and reasons!


In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
# Test the tools directly to make sure they work
print("🔧 Testing tools directly:")

# Test text search tool
result = text_search_tool("furry visual novels", num_results=5)
print("Text search result:")
print(result[:200] + "..." if len(result) > 200 else result)

🔧 Testing tools directly:
🔍 Searching for: 'furry visual novels'
Text search result:
Found 4 games matching 'furry visual novels':

1. 🎮 Burrow of the Fallen Bear: A Gay Furry Visual Novel (Score: 1.00)
📝 None...
⭐ Critics: Burrow of the Fallen Bear é uma visual novel honesta quanto à...


In [ ]:
# Test the agent with a text query
query1 = "I want to find games like The Witcher 3 with fantasy RPG elements"

print("🎮 Testing Text Search Tool:")
print(f"Query: {query1}")
print("\nAgent's response:")
print("-" * 30)

response = await agent.run(query1)
print(str(response))

🎮 Testing Text Search Tool:
Query: I want to find games like The Witcher 3 with fantasy RPG elements

Agent's response:
------------------------------
Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The Witcher 3 is a fantasy RPG game, so games with fantasy RPG elements include *The Witcher 3: Wild Hunt*, *Dungeons & Dragons: The Elder Scrolls*, and *Mass Effect: War of the Worlds*.


🔍 Searching for: 'indie games with unique art styles'


In [12]:
# Test 2: Complex query with reasoning
print("\n🧪 Test 2: Complex Query with Reasoning")
print("=" * 50)

query2 = "I'm looking for game that have lots of cute animals. I like cats and dogs, but I don't like humans to be in the game."
print(f"Query: {query2}")
print("\nAgent's response:")
print("-" * 30)

response2 = await agent.run(query2)
print(str(response2))


🧪 Test 2: Complex Query with Reasoning
Query: I'm looking for game that have lots of cute animals. I like cats and dogs, but I don't like humans to be in the game.

Agent's response:
------------------------------
Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
I'm sorry, but I can't answer the question with the provided tools. The tools available are text_search and image_search, which are designed to search for games based on text descriptions or visual similarities. I need more information to help with your query.


In [13]:
import asyncio

print("🎮 Notebook-Friendly Demo - Test with Predefined Queries!")
print("=" * 60)

# Predefined test queries
test_queries = [
    "Find me puzzle games that are relaxing",
    "I want NSFW games but not too violent", 
    "Show me indie games with unique art styles",
    "I'm looking for multiplayer games for 2-4 players"
]

print("🧪 Testing with predefined queries:\n")

for i, query in enumerate(test_queries, 1):
    print(f"Test {i}: {query}")
    print("-" * 40)
    try:
        response = asyncio.run(agent.run(query))
        print(str(response))
    except Exception as e:
        print(f"❌ Error: {e}")
    print("\n" + "="*60 + "\n")

🎮 Notebook-Friendly Demo - Test with Predefined Queries!
🧪 Testing with predefined queries:

Test 1: Find me puzzle games that are relaxing
----------------------------------------
Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
Puzzle games that are relaxing include games like "Pac-Man" (which is a classic puzzle game), "Minecraft" (with its creative elements), and "Sudoku" (which is relaxing and logical). Let me know if you'd like more details!


Test 2: I want NSFW games but not too violent
----------------------------------------
Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step pars

## Step 7: Understanding How Your Agent Works

### Let's break down what just happened:

1. **Query Analysis**: The agent understood what you were looking for
2. **Tool Selection**: It chose the right tool(s) to use
3. **Execution**: It ran the search and got results
4. **Reasoning**: It analyzed the results and made recommendations
5. **Response**: It formatted a helpful answer

### Key Benefits of Your Agent:
- **Multi-modal**: Can handle both text and image queries
- **Intelligent**: Uses reasoning to understand complex requests
- **Flexible**: Can combine multiple tools for better results
- **Explainable**: Shows its reasoning process
- **Local**: Everything runs on your machine


## Step 8: Advanced Features & Next Steps

### What You've Built:
✅ **Multi-modal Agent**: Handles text and image queries
✅ **Intelligent Reasoning**: Uses ReAct pattern for complex queries
✅ **Local Processing**: Everything runs on your machine
✅ **Tool Integration**: Seamlessly uses your existing RAG system

### Next Steps to Improve Your Agent:

1. **Better LLM**: Replace mock LLM with Ollama
2. **More Tools**: Add genre filtering, platform filtering, etc.
3. **Memory**: Add conversation memory
4. **Custom Prompts**: Fine-tune the agent's behavior
5. **Evaluation**: Test with real users and improve

### Advanced Features to Add:
- **Conversation Memory**: Remember previous queries
- **User Preferences**: Learn from user feedback
- **Multi-step Planning**: Break down complex requests
- **Explanation Generation**: Better reasoning explanations
- **Result Ranking**: Learn from user interactions


In [16]:
# Next Steps: Upgrade to Real LLM
print("🚀 Next Steps to Improve Your Agent:")
print("=" * 50)

print("\n1. 🔧 Install Ollama and Load a Real LLM:")
print("   - Install: https://ollama.ai/download")
print("   - Run: ollama pull llama2:7b")
print("   - Replace MockLLM with: llm = Ollama(model='llama2:7b')")

print("\n2. 🛠️ Add More Tools:")
print("   - Genre filtering tool")
print("   - Platform filtering tool")
print("   - Year range filtering tool")
print("   - User rating filtering tool")

print("\n3. 🧠 Add Memory:")
print("   - Conversation history")
print("   - User preferences")
print("   - Learning from feedback")

print("\n4. 🎯 Customize Behavior:")
print("   - Custom system prompts")
print("   - Domain-specific instructions")
print("   - Response formatting")

print("\n5. 📊 Evaluate and Improve:")
print("   - Test with real users")
print("   - Collect feedback")
print("   - Iterate and improve")

print("\n🎉 Congratulations! You've built your first Agentic RAG system!")
print("\nThis is just the beginning. You can now:")
print("  - Deploy it as a web service")
print("  - Add it to your existing applications")
print("  - Extend it with more features")
print("  - Share it with others!")

print("\n🤔 Questions? Check out the LlamaIndex documentation:")
print("   https://docs.llamaindex.ai/")
print("\nHappy coding! 🚀")


🚀 Next Steps to Improve Your Agent:

1. 🔧 Install Ollama and Load a Real LLM:
   - Install: https://ollama.ai/download
   - Run: ollama pull llama2:7b
   - Replace MockLLM with: llm = Ollama(model='llama2:7b')

2. 🛠️ Add More Tools:
   - Genre filtering tool
   - Platform filtering tool
   - Year range filtering tool
   - User rating filtering tool

3. 🧠 Add Memory:
   - Conversation history
   - User preferences
   - Learning from feedback

4. 🎯 Customize Behavior:
   - Custom system prompts
   - Domain-specific instructions
   - Response formatting

5. 📊 Evaluate and Improve:
   - Test with real users
   - Collect feedback
   - Iterate and improve

🎉 Congratulations! You've built your first Agentic RAG system!

This is just the beginning. You can now:
  - Deploy it as a web service
  - Add it to your existing applications
  - Extend it with more features
  - Share it with others!

🤔 Questions? Check out the LlamaIndex documentation:
   https://docs.llamaindex.ai/

Happy coding! 🚀
